## Python loading

In [149]:
import sys

import pandas as pd
from pandas import DataFrame, Series

import numpy as np

import matplotlib.pyplot as plt
import matplotlib

#import xgboost as xgb

from sklearn import tree
from sklearn.ensemble import RandomForestClassifier 

%matplotlib inline

In [150]:
train = pd.read_csv("../../data/train.csv", index_col=0)
test = pd.read_csv("../../data/test.csv", index_col=0)

train.head(2)

,Name,DateTime,OutcomeType,OutcomeSubtype,AnimalType,SexuponOutcome,AgeuponOutcome,Breed,Color
AnimalID,,,,,,,,,
A671945,Hambone,2014-02-12 18:22:00,Return_to_owner,NaN,Dog,Neutered Male,1 year,Shetland Sheepdog Mix,Brown/White
A656520,Emily,2013-10-13 12:44:00,Euthanasia,Suffering,Cat,Spayed Female,1 year,Domestic Shorthair Mix,Cream Tabby


## Data fitting

In [139]:
train.SexuponOutcome.value_counts()

Neutered Male    9779
Spayed Female    8820
Intact Male      3525
Intact Female    3511
Unknown          1093
Name: SexuponOutcome, dtype: int64

In [151]:
opt = train.copy()

opt["Intact"] = opt.SexuponOutcome.replace("Intact Male", "Yes")
opt.Intact = opt.Intact.replace("Neutered Male", "No")
opt.Intact = opt.Intact.replace("Spayed Female", "No")
opt.Intact = opt.Intact.replace("Intact Female", "Yes")
opt.Intact = opt.Intact.replace("Unknown", np.nan)

opt.Intact = opt.Intact.astype("category")

opt.SexuponOutcome = opt.SexuponOutcome.replace("Neutered Male", "Male")
opt.SexuponOutcome = opt.SexuponOutcome.replace("Intact Male", "Male")
opt.SexuponOutcome = opt.SexuponOutcome.replace("Spayed Female", "Female")
opt.SexuponOutcome = opt.SexuponOutcome.replace("Intact Female", "Female")
opt.SexuponOutcome = opt.SexuponOutcome.replace("Unknown", np.nan)

opt.SexuponOutcome = opt.SexuponOutcome.astype("category")

test.SexuponOutcome = test.SexuponOutcome.replace("Neutered Male", "Male")
test.SexuponOutcome = test.SexuponOutcome.replace("Intact Male", "Male")
test.SexuponOutcome = test.SexuponOutcome.replace("Spayed Female", "Female")
test.SexuponOutcome = test.SexuponOutcome.replace("Intact Female", "Female")
test.SexuponOutcome = test.SexuponOutcome.replace("Unknown", np.nan)

test.SexuponOutcome = test.SexuponOutcome.astype("category")



opt.SexuponOutcome.value_counts()
    
#opt.Breed.value_counts()
#opt

Male      13304
Female    12331
dtype: int64

In [141]:
opt.Color.value_counts()

opt[(opt.Color == "Blue") & (opt.AnimalType == "Dog")].Breed.value_counts()

Pit Bull Mix                     34
Weimaraner Mix                    4
Labrador Retriever Mix            3
Blue Lacy Mix                     3
Weimaraner                        3
Chihuahua Shorthair Mix           2
Pit Bull                          2
Cane Corso Mix                    1
Australian Kelpie/Blue Lacy       1
Chow Chow                         1
Great Dane Mix                    1
Staffordshire Mix                 1
Australian Kelpie Mix             1
Great Dane/Pit Bull               1
Chinese Sharpei/Great Dane        1
Chinese Sharpei                   1
Pit Bull/Flat Coat Retriever      1
Australian Shepherd/Chow Chow     1
Pit Bull/Weimaraner               1
Blue Lacy/Australian Kelpie       1
Standard Schnauzer Mix            1
Name: Breed, dtype: int64

In [152]:
opt.AnimalType = opt.AnimalType.astype("category")
opt.AnimalType.value_counts()

Dog    15595
Cat    11134
dtype: int64

In [153]:
opt.OutcomeType = opt.OutcomeType.astype("category")
opt.OutcomeType.value_counts()


Adoption           10769
Transfer            9422
Return_to_owner     4786
Euthanasia          1555
Died                 197
dtype: int64

## Prediction

In [179]:
#create and train the random forest
    #multi-core CPUs can use: rf = RandomForestClassifier(n_estimators=100, n_jobs=2)
rf = RandomForestClassifier(n_estimators=100, n_jobs=2)

d_train = opt.SexuponOutcome.cat.codes.values
d_target = opt.OutcomeType.cat.codes.values

d_target
d_train

rf.fit(d_train.reshape(-1, 1), d_target)

#opt.SexuponOutcome.cat.categories.values

#test.SexuponOutcome

test.SexuponOutcome = test.SexuponOutcome.astype("category")

#rf.predict(4)

ii = 0
for i in test.SexuponOutcome.cat.codes.values:
    ii = ii + 1
    print(i)
    print(rf.predict(i))
    
    if ii > 50:
        break


0
[0]
0
[0]
1
[0]
1
[0]
1
[0]
1
[0]
1
[0]
0
[0]
0
[0]
0
[0]
0
[0]
0
[0]
0
[0]
1
[0]
1
[0]
0
[0]
1
[0]
1
[0]
0
[0]
0
[0]
0
[0]
